# Stage 3: 1cSVM for Anomaly Detection for IoT 7 and 8
The 1cSVM  anomaly detection model used in this project work as follows:
## 1. We  compressed the data using  AE model. The compressed  data  for devise 7&8 from AE output  was used as input data for 1cSVM.
## 2. The data was split into train and test and put through 1cSVM and a confusion matrix was drawn.
##  3. After testing  a new device [9] was put though the model.

In [1]:
from mmae.multimodal_autoencoder import MultimodalAutoencoder

from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.callbacks import EarlyStopping
from keras import backend, Model, regularizers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans

from scipy import spatial

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy.random import seed
seed(42)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

Using TensorFlow backend.


In [2]:
# Import and stitch all benign data together

device7 = pd.read_csv("7.benign.csv")
device8 = pd.read_csv("8.benign.csv")


all_benign = pd.concat([device7,device8])
all_benign['label'] = 0

cut_off = int(all_benign.shape[0]/2)

# Split the benign data into train and test
all_benign, all_benign_test,_,_ = train_test_split(all_benign.iloc[:,:-1].values, 
                                all_benign.iloc[:,-1].values, test_size=0.5, random_state=42)

# Scale the train data
scaler = StandardScaler()
scaler.fit(all_benign)
all_benign = scaler.transform(all_benign)

print(all_benign.shape)

(49367, 115)


In [3]:

# function to calculate rmse
def rmse(y_true, y_pred):
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

# Auto Encoder inout layer
input_layer = Input(shape=(all_benign.shape[1],))

# Encoder part
encoded = Dense(110, activation='relu', activity_regularizer=regularizers.l1_l2(l1=10e-6, l2=10e-6))(input_layer)
encoded = Dense(95, activation='relu')(encoded)
encoded = Dense(20)(encoded)

# Decoder part
decoded = Dense(95, activation='relu')(encoded)
decoded = Dense(110, activation='relu')(decoded)

# Output layer
output_layer = Dense(all_benign.shape[1])(decoded)

# Compile autoencoder
autoencoder = Model(input_layer, output_layer)
encoder = Model(input_layer, encoded)
autoencoder.compile(optimizer= 'adadelta', loss=rmse)

# fit the model
autoencoder.fit(all_benign, all_benign, batch_size = 100, epochs = 450,
                validation_split = 0.20, callbacks=[EarlyStopping(patience=50,monitor='val_loss')])

# predict on validation and other benign 
benign1_test = encoder.predict(all_benign)

Train on 39493 samples, validate on 9874 samples
Epoch 1/450
39493/39493 [==============================] - 2s 44us/step - loss: 0.3583 - val_loss: 0.2539
Epoch 2/450
39493/39493 [==============================] - 1s 38us/step - loss: 0.2134 - val_loss: 0.1899
Epoch 3/450
39493/39493 [==============================] - 2s 38us/step - loss: 0.1705 - val_loss: 0.1503
Epoch 4/450
39493/39493 [==============================] - 2s 40us/step - loss: 0.1471 - val_loss: 0.1325
Epoch 5/450
39493/39493 [==============================] - 2s 39us/step - loss: 0.1328 - val_loss: 0.1290
Epoch 6/450
39493/39493 [==============================] - 1s 36us/step - loss: 0.1235 - val_loss: 0.1096
Epoch 7/450
39493/39493 [==============================] - 1s 37us/step - loss: 0.1161 - val_loss: 0.1086
Epoch 8/450
39493/39493 [==============================] - 1s 36us/step - loss: 0.1109 - val_loss: 0.1060
Epoch 9/450
39493/39493 [==============================] - 2s 40us/step - loss: 0.1061 - val_loss: 0.09

39493/39493 [==============================] - 1s 35us/step - loss: 0.0467 - val_loss: 0.0429
Epoch 154/450
39493/39493 [==============================] - 1s 36us/step - loss: 0.0468 - val_loss: 0.0492
Epoch 155/450
39493/39493 [==============================] - 1s 34us/step - loss: 0.0465 - val_loss: 0.0473
Epoch 156/450
39493/39493 [==============================] - 1s 34us/step - loss: 0.0469 - val_loss: 0.0497
Epoch 157/450
39493/39493 [==============================] - 1s 35us/step - loss: 0.0468 - val_loss: 0.0488
Epoch 158/450
39493/39493 [==============================] - 2s 40us/step - loss: 0.0466 - val_loss: 0.0457
Epoch 159/450
39493/39493 [==============================] - 2s 45us/step - loss: 0.0465 - val_loss: 0.0484
Epoch 160/450
39493/39493 [==============================] - 1s 37us/step - loss: 0.0463 - val_loss: 0.0460
Epoch 161/450
39493/39493 [==============================] - 1s 37us/step - loss: 0.0464 - val_loss: 0.0459
Epoch 162/450
39493/39493 [===============

Epoch 304/450
39493/39493 [==============================] - 1s 37us/step - loss: 0.0413 - val_loss: 0.0471
Epoch 305/450
39493/39493 [==============================] - 1s 34us/step - loss: 0.0415 - val_loss: 0.0422
Epoch 306/450
39493/39493 [==============================] - 2s 41us/step - loss: 0.0417 - val_loss: 0.0403
Epoch 307/450
39493/39493 [==============================] - 1s 34us/step - loss: 0.0415 - val_loss: 0.0466
Epoch 308/450
39493/39493 [==============================] - 1s 36us/step - loss: 0.0412 - val_loss: 0.0404
Epoch 309/450
39493/39493 [==============================] - 2s 41us/step - loss: 0.0415 - val_loss: 0.0409
Epoch 310/450
39493/39493 [==============================] - 2s 39us/step - loss: 0.0412 - val_loss: 0.0393
Epoch 311/450
39493/39493 [==============================] - 1s 36us/step - loss: 0.0414 - val_loss: 0.0387
Epoch 312/450
39493/39493 [==============================] - 1s 34us/step - loss: 0.0413 - val_loss: 0.0439
Epoch 313/450
39493/39493 [=

Epoch 380/450
39493/39493 [==============================] - 1s 36us/step - loss: 0.0402 - val_loss: 0.0426
Epoch 381/450
39493/39493 [==============================] - 2s 40us/step - loss: 0.0401 - val_loss: 0.0388
Epoch 382/450
39493/39493 [==============================] - 2s 40us/step - loss: 0.0401 - val_loss: 0.0402
Epoch 383/450
39493/39493 [==============================] - 1s 35us/step - loss: 0.0402 - val_loss: 0.0391
Epoch 384/450
39493/39493 [==============================] - 1s 35us/step - loss: 0.0402 - val_loss: 0.0402
Epoch 385/450
39493/39493 [==============================] - 2s 41us/step - loss: 0.0404 - val_loss: 0.0406
Epoch 386/450
39493/39493 [==============================] - 1s 34us/step - loss: 0.0402 - val_loss: 0.0403
Epoch 387/450
39493/39493 [==============================] - 1s 35us/step - loss: 0.0400 - val_loss: 0.0434
Epoch 388/450
39493/39493 [==============================] - 1s 34us/step - loss: 0.0400 - val_loss: 0.0434
Epoch 389/450
39493/39493 [=

In [4]:
# Read and combine anomaly data
anomaly = pd.concat([pd.read_csv("7.gafgyt.combo.csv"), pd.read_csv("7.gafgyt.junk.csv"), pd.read_csv("7.gafgyt.scan.csv"),pd.read_csv("7.gafgyt.tcp.csv"), pd.read_csv("7.gafgyt.udp.csv"),pd.read_csv("8.gafgyt.combo.csv"), pd.read_csv("8.gafgyt.junk.csv"), pd.read_csv("8.gafgyt.scan.csv"),pd.read_csv("8.gafgyt.tcp.csv"), pd.read_csv("8.gafgyt.udp.csv"), pd.read_csv("8.mirai.ack.csv"),pd.read_csv("8.mirai.scan.csv"), pd.read_csv("8.mirai.syn.csv"), pd.read_csv("8.mirai.udp.csv"), pd.read_csv("8.mirai.udpplain.csv")])

column_names = [anomaly .columns.values]



In [5]:
# Scale and label anomaly data
anomaly = scaler.transform(anomaly)
anomaly = pd.DataFrame(anomaly,columns = column_names)
anomaly['label'] = 1

# Scale and label benign test data
all_benign_test = scaler.transform(all_benign_test)
all_benign_test = pd.DataFrame(all_benign_test, columns = column_names)
all_benign_test['label'] = 0

#get the intial training data as well
benign1_test = pd.DataFrame(benign1_test)
benign1_test['label'] = 0

print(anomaly.shape)
print(all_benign_test.shape)
print(benign1_test.shape)

# Combine benign and anomaly data
test_data = pd.concat([anomaly,all_benign_test])

test_data = test_data.sample(frac=1, random_state=42)

compressed_data = encoder.predict(test_data.iloc[:,:-1])
compressed_data = pd.DataFrame(compressed_data)
compressed_data['label'] = test_data.iloc[:,-1].values
compressed_data = pd.concat([compressed_data,benign1_test])

del all_benign_test, anomaly, benign1_test, test_data

print(compressed_data.head())
print(compressed_data.tail())
print(compressed_data.shape)



(1139543, 116)
(49368, 116)
(49367, 21)
            0           1          2           3           4           5  \
0  -64.640724  -75.699188  36.760426   13.909013 -125.218208   38.657703   
1   -7.117319   -1.907917  -1.311013   -0.943912    0.062029    2.798764   
2 -175.567108 -172.088898   5.381436 -123.033615  -47.375862  215.157684   
3   -7.117319   -1.907917  -1.311013   -0.943912    0.062029    2.798764   
4 -112.418404 -124.301582   1.696837  -86.563072  -16.310423  147.387589   

           6           7           8           9         10          11  \
0  11.824189 -265.506683  203.832382 -120.234169 -90.639069  119.334579   
1   4.810143   -0.267949    0.689625   -0.686367   5.473677   -2.049598   
2  -1.728666 -207.461792  204.222321  -41.723034  -1.197372   44.859261   
3   4.810143   -0.267949    0.689625   -0.686367   5.473677   -2.049598   
4  -2.905084 -138.065674  146.273712  -38.438118   3.463380   33.617290   

          12         13         14         15       

In [6]:
# seperate out benign and anomaly data
benign_78 = compressed_data.loc[compressed_data['label'] == 0]
anomaly_78 = compressed_data.loc[compressed_data['label'] == 1]

benign_78.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,label
20,-0.073150,-0.148070,-0.006054,-0.177787,-0.211695,-0.035594,0.184644,0.199323,0.170821,-0.140831,-0.004633,-0.052033,-0.226290,-0.051841,0.068216,0.206028,0.131335,-0.136266,0.214769,-0.066641,0
89,-0.201085,0.210977,0.250692,-0.187757,0.033367,-0.119585,-0.468449,0.199692,0.129238,0.108742,0.331348,-0.184035,0.109604,0.367551,0.207928,-0.165404,-0.579933,-0.146664,0.289422,-0.141094,0
132,-2.869385,0.499699,3.363173,1.307569,-1.766352,0.990319,-0.756660,-0.285249,-1.147055,1.716674,-1.675314,1.516451,0.503657,-1.301855,-6.930199,-0.059917,-1.691387,-1.194737,2.384669,1.084768,0
150,-0.047595,0.271234,-0.361656,0.152302,0.155801,-0.378130,-0.900460,-0.192904,0.011110,0.173823,0.497745,-0.604379,0.299349,0.075340,0.233428,-0.140827,-0.575089,-0.348442,0.061787,0.210615,0
169,-0.042211,0.225937,0.119994,0.054492,0.120763,0.003177,0.159740,-0.228109,-0.038167,-0.084716,-0.110239,0.007555,0.046120,0.102639,0.127427,-0.141997,0.095362,-0.050423,0.126247,0.140065,0


In [7]:
benign_78.shape

(98735, 21)

In [8]:
print(anomaly_78.head())
anomaly_78.shape

            0           1          2           3           4           5  \
0  -64.640724  -75.699188  36.760426   13.909013 -125.218208   38.657703   
1   -7.117319   -1.907917  -1.311013   -0.943912    0.062029    2.798764   
2 -175.567108 -172.088898   5.381436 -123.033615  -47.375862  215.157684   
3   -7.117319   -1.907917  -1.311013   -0.943912    0.062029    2.798764   
4 -112.418404 -124.301582   1.696837  -86.563072  -16.310423  147.387589   

           6           7           8           9         10          11  \
0  11.824189 -265.506683  203.832382 -120.234169 -90.639069  119.334579   
1   4.810143   -0.267949    0.689625   -0.686367   5.473677   -2.049598   
2  -1.728666 -207.461792  204.222321  -41.723034  -1.197372   44.859261   
3   4.810143   -0.267949    0.689625   -0.686367   5.473677   -2.049598   
4  -2.905084 -138.065674  146.273712  -38.438118   3.463380   33.617290   

          12         13         14         15          16         17  \
0 -21.979618  90.777

(1139543, 21)

In [9]:
# remove the label from benign data
benign_78 = benign_78.drop(['label'], axis=1)
benign_78.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
20,-0.073150,-0.148070,-0.006054,-0.177787,-0.211695,-0.035594,0.184644,0.199323,0.170821,-0.140831,-0.004633,-0.052033,-0.226290,-0.051841,0.068216,0.206028,0.131335,-0.136266,0.214769,-0.066641
89,-0.201085,0.210977,0.250692,-0.187757,0.033367,-0.119585,-0.468449,0.199692,0.129238,0.108742,0.331348,-0.184035,0.109604,0.367551,0.207928,-0.165404,-0.579933,-0.146664,0.289422,-0.141094
132,-2.869385,0.499699,3.363173,1.307569,-1.766352,0.990319,-0.756660,-0.285249,-1.147055,1.716674,-1.675314,1.516451,0.503657,-1.301855,-6.930199,-0.059917,-1.691387,-1.194737,2.384669,1.084768
150,-0.047595,0.271234,-0.361656,0.152302,0.155801,-0.378130,-0.900460,-0.192904,0.011110,0.173823,0.497745,-0.604379,0.299349,0.075340,0.233428,-0.140827,-0.575089,-0.348442,0.061787,0.210615
169,-0.042211,0.225937,0.119994,0.054492,0.120763,0.003177,0.159740,-0.228109,-0.038167,-0.084716,-0.110239,0.007555,0.046120,0.102639,0.127427,-0.141997,0.095362,-0.050423,0.126247,0.140065


In [10]:
# train test split on benign data into 50% train and 50% test
all_benign_train78, all_benign_test78  = train_test_split(benign_78, test_size=0.5, random_state=42)


In [11]:
# fit a one class SVM on benign train data
from sklearn.svm import OneClassSVM

oneclass78 = OneClassSVM(kernel='rbf', gamma=.02,nu=0.02).fit(all_benign_train78)



In [12]:
# create test data

# change the all_benign_test label to 1, create the label dataframe and drop the label from original dataframe
all_benign_test78['label'] = 0
benign_label78 = all_benign_test78['label'] + 1
all_benign_test78 = all_benign_test78.drop(['label'], axis=1)

# change the anomlay label to -1, create the label dataframe and drop the label from original dataframe
anomaly_label78 = anomaly_78['label']-2
anomaly_78 = anomaly_78.drop(['label'], axis=1)

# append benign and anonlay data to create overall test data and test label
test_data78 = all_benign_test78.append(anomaly_78)
test_label78 = benign_label78.append(anomaly_label78)

# rune the one class svm prediction on test data
prediction78 = oneclass78.predict(test_data78)

D:\Users\608777\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [13]:
# confusion matrix

cm = confusion_matrix(test_label78 , prediction78)
cm

array([[1139339,     204],
       [   1048,   48320]], dtype=int64)

In [14]:
from sklearn.metrics import classification_report
print(classification_report(test_label78 , prediction78))

              precision    recall  f1-score   support

          -1       1.00      1.00      1.00   1139543
           1       1.00      0.98      0.99     49368

    accuracy                           1.00   1188911
   macro avg       1.00      0.99      0.99   1188911
weighted avg       1.00      1.00      1.00   1188911



In [15]:
#Confusion matrix - Percentage
cm = pd.DataFrame(confusion_matrix(test_label78 , prediction78,normalize = 'true',labels=[-1,1]),index=['0', '1'],columns = ['0', '1'])
cm

,0,1
0,0.999821,0.000179
1,0.021228,0.978772
